In [1]:
print(1)

1


# 第一问

In [14]:
import numpy as np

def solve_problem_1_multipoint():
    """
    解决数学建模竞赛的问题1（高精度版）。
    使用个关键目标点来判断遮蔽，计算单枚烟幕弹对导弹M1的有效遮蔽时长。
    """

    # --- 1. 常量与初始条件 (与之前相同) ---

    G = 9.8
    P_M1_INITIAL = np.array([20000.0, 0.0, 2000.0])
    P_FY1_INITIAL = np.array([17800.0, 0.0, 1800.0])
    P_FAKE_TARGET = np.array([0.0, 0.0, 0.0])
    V_M1_SPEED = 300.0
    V_FY1_SPEED = 120.0
    T_DROP = 1.5
    T_EXPLODE_AFTER_DROP = 3.6
    CLOUD_RADIUS = 10.0
    CLOUD_SINK_SPEED = 3.0
    CLOUD_EFFECTIVE_DURATION = 20.0
    
    # --- 新增：定义5个关键目标点 ---
    # 目标半径为7m, 高度为10m, 中心在 y=200, z=5
    P_TRUE_TARGET_POINTS = [
        # 1. 中心点
        np.array([0.0, 200.0, 5.0]),  # Center
        # 2. 轮廓最接近导弹的顶部和底部
        np.array([0.0, 193.0, 10.0]), # Closest-Top (y=200-7)
        np.array([0.0, 193.0, 0.0]),  # Closest-Bottom
        # 3. 轮廓最远离导弹的顶部和底部
        np.array([0.0, 207.0, 5.0]), # Farthest-Top (y=200+7)
        np.array([0.0, 200.0, 10.0]), # Farthest-Middle (y=200)
        np.array([0.0, 207.0, 10.0]), # Farthest-Top (y=200+7)
        np.array([0.0, 207.0, 0.0]),  # Farthest-Bottom
        np.array([0.0, 193.0, 5.0]),   # Closest-Middle
        np.array([0.0, 195.0, 5.0])    # Closest-Middle
    ]


    # --- 2. 关键事件与向量的预计算 (与之前相同) ---

    V_FY1 = np.array([-V_FY1_SPEED, 0.0, 0.0])
    P_DROP = P_FY1_INITIAL + V_FY1 * T_DROP
    T_DETONATION = T_DROP + T_EXPLODE_AFTER_DROP
    x_det = P_DROP[0] + V_FY1[0] * T_EXPLODE_AFTER_DROP
    y_det = P_DROP[1]
    z_det = P_DROP[2] - 0.5 * G * T_EXPLODE_AFTER_DROP**2
    P_DETONATION = np.array([x_det, y_det, z_det])
    direction_m1 = P_FAKE_TARGET - P_M1_INITIAL
    V_M1 = (V_M1_SPEED / np.linalg.norm(direction_m1)) * direction_m1


    # --- 3. 动态位置与遮蔽计算函数 (核心修改部分) ---

    def get_missile_pos(t):
        return P_M1_INITIAL + V_M1 * t

    def get_cloud_center_pos(t):
        time_since_detonation = t - T_DETONATION
        sink_distance = CLOUD_SINK_SPEED * time_since_detonation
        return P_DETONATION - np.array([0.0, 0.0, sink_distance])

    def is_fully_shielded(t):
        """
        判断在t时刻，是否所有关键目标点的视线都被同时遮蔽。
        """
        missile_pos = get_missile_pos(t)
        cloud_pos = get_cloud_center_pos(t)
        vec_missile_to_cloud = cloud_pos - missile_pos

        # 遍历每一个目标点
        for target_point in P_TRUE_TARGET_POINTS:
            vec_missile_to_target = target_point - missile_pos
            
            # 计算云团中心到该视线的距离
            cross_product_magnitude = np.linalg.norm(np.cross(vec_missile_to_cloud, vec_missile_to_target))
            line_vector_magnitude = np.linalg.norm(vec_missile_to_target)
            
            if line_vector_magnitude == 0: continue
            
            distance = cross_product_magnitude / line_vector_magnitude
            
            # 只要有一个点的视线没有被遮蔽，就立刻返回False
            if distance > CLOUD_RADIUS:
                return False
        
        # 如果循环结束，所有点都被遮蔽，则返回True
        return True


    # --- 4. 通过仿真循环求解遮蔽时长 ---

    time_step = 0.001
    start_time = T_DETONATION
    end_time = T_DETONATION + CLOUD_EFFECTIVE_DURATION
    
    total_shielding_time = 0.0
    shield_start_time = -1.0
    shield_end_time = -1.0
    
    was_shielded_previously = False

    for t in np.arange(start_time, end_time, time_step):
        # 调用新的判断函数
        is_shielded_now = is_fully_shielded(t)
        
        if is_shielded_now:
            total_shielding_time += time_step
        
        if is_shielded_now and not was_shielded_previously:
            shield_start_time = t
        elif not is_shielded_now and was_shielded_previously:
            # 记录第一次遮蔽结束的时间
            if shield_end_time == -1.0:
                 shield_end_time = t

        was_shielded_previously = is_shielded_now

    # --- 5. 输出结果 ---
    
    print("--- 求解结果：问题1 (多点高精度模型) ---")
    print(f"\n模型假设:")
    print(f" 采用多个关键点定义真目标轮廓，要求视线被完全遮蔽。")

    print(f"\n关键事件参数:")
    print(f" 烟幕弹引爆时间: {T_DETONATION:.3f} s")
    print(f" 烟幕弹引爆点坐标: ({P_DETONATION[0]:.3f}, {P_DETONATION[1]:.3f}, {P_DETONATION[2]:.3f})")

    print(f"\n遮蔽效果分析:")
    if total_shielding_time > 0:
        print(f" 有效遮蔽开始于: t ≈ {shield_start_time:.3f} s")
        print(f" 有效遮蔽结束于: t ≈ {shield_end_time:.3f} s")
        print(f"\n 最终结果：有效遮蔽总时长 ≈ {total_shielding_time:.3f} s")
    else:
        print(" 在此更严格的条件下，未能形成有效的完全遮蔽。")

# 运行求解函数
solve_problem_1_multipoint()

--- 求解结果：问题1 (多点高精度模型) ---

模型假设:
 采用多个关键点定义真目标轮廓，要求视线被完全遮蔽。

关键事件参数:
 烟幕弹引爆时间: 5.100 s
 烟幕弹引爆点坐标: (17188.000, 0.000, 1736.496)

遮蔽效果分析:
 有效遮蔽开始于: t ≈ 8.057 s
 有效遮蔽结束于: t ≈ 11.967 s

 最终结果：有效遮蔽总时长 ≈ 3.910 s


# 第二题

In [14]:
import numpy as np
import time

# --- 1. 定义常量 (已根据题目核对) ---
# 导弹M1
V_M1 = 300  # 速度 (m/s)
P0_M1 = np.array([20000, 0, 2000])  # 初始位置 (m)
TARGET_FALSE = np.array([0, 0, 0])   # 假目标位置 (m)

# 无人机 FY1
P0_FY1 = np.array([17800, 0, 1800]) # 初始位置 (m)
V_FY1_MIN, V_FY1_MAX = 70, 140       # 速度范围 (m/s)

# 真目标 (圆柱体)
TARGET_TRUE_CENTER_BASE = np.array([0, 200, 0]) # 底面中心
TARGET_TRUE_RADIUS = 7    # 半径 (m)
TARGET_TRUE_HEIGHT = 10   # 高度 (m)

# 烟幕
SMOKE_CLOUD_RADIUS = 10   # 烟幕云半径 (m)
SMOKE_CLOUD_SINK_V = 3    # 烟幕云下沉速度 (m/s)
SMOKE_DURATION = 20       # 烟幕有效持续时间 (s)

# 物理常量
G = 9.8  # 重力加速度 (m/s^2)

# --- 2. 运动学模型 ---
direction_m1 = (TARGET_FALSE - P0_M1) / np.linalg.norm(TARGET_FALSE - P0_M1)
def missile_position(t):
    return P0_M1 + V_M1 * t * direction_m1

def uav_position(v_f, theta, t):
    vx = v_f * np.cos(theta)
    vy = v_f * np.sin(theta)
    return P0_FY1 + t * np.array([vx, vy, 0])

def grenade_position(v_f, theta, t_d, t):
    if t < t_d: return uav_position(v_f, theta, t)
    p0_grenade = uav_position(v_f, theta, t_d)
    v0_grenade = np.array([v_f * np.cos(theta), v_f * np.sin(theta), 0])
    dt = t - t_d
    pos = p0_grenade + v0_grenade * dt + 0.5 * np.array([0, 0, -G]) * dt**2
    return pos

def cloud_center_position(v_f, theta, t_d, t_b, t):
    if t < t_b: return None
    p_detonation = grenade_position(v_f, theta, t_d, t_b)
    dt = t - t_b
    return p_detonation + dt * np.array([0, 0, -SMOKE_CLOUD_SINK_V])

# --- 3. 几何遮蔽判断 ---
target_key_points = []
for h in [0, TARGET_TRUE_HEIGHT]:
    center = TARGET_TRUE_CENTER_BASE + np.array([0, 0, h])
    for angle in np.linspace(0, 2 * np.pi, 8, endpoint=False): # 增加关键点以提高精度
        target_key_points.append(center + np.array([TARGET_TRUE_RADIUS * np.cos(angle), TARGET_TRUE_RADIUS * np.sin(angle), 0]))
target_key_points.append(TARGET_TRUE_CENTER_BASE + np.array([0, 0, TARGET_TRUE_HEIGHT/2]))

def is_line_segment_intercepted_by_sphere(p1, p2, sphere_center, sphere_radius):
    v = p2 - p1
    a = np.dot(v, v)
    if a == 0: return False
    b = 2 * np.dot(v, p1 - sphere_center)
    c = np.dot(p1 - sphere_center, p1 - sphere_center) - sphere_radius**2
    discriminant = b**2 - 4*a*c
    if discriminant < 0: return False
    t1 = (-b - np.sqrt(discriminant)) / (2 * a)
    t2 = (-b + np.sqrt(discriminant)) / (2 * a)
    if (0 <= t1 <= 1) or (0 <= t2 <= 1) or (t1*t2 < 0): return True
    return False

def is_shielded(m_pos, c_pos):
    if c_pos is None: return False
    for point in target_key_points:
        if not is_line_segment_intercepted_by_sphere(m_pos, point, c_pos, SMOKE_CLOUD_RADIUS):
            return False
    return True

# --- 4. 适应度函数 ---
def calculate_fitness(params):
    v_f, theta, t_d, t_b = params
    if t_d < 1.5 or t_b <= t_d: return 0.0
    total_shielding_time = 0
    time_step = 0.01 # 使用更小的时间步长以提高精度
    for t in np.arange(t_b, t_b + SMOKE_DURATION, time_step):
        m_pos = missile_position(t)
        if m_pos[0] <= TARGET_TRUE_CENTER_BASE[0]: break
        c_pos = cloud_center_position(v_f, theta, t_d, t_b, t)
        if c_pos is not None and c_pos[2] < -SMOKE_CLOUD_RADIUS: break
        if is_shielded(m_pos, c_pos):
            total_shielding_time += time_step
    return total_shielding_time

# --- 5. PSO 主算法 ---
def pso_optimizer(n_particles, n_iterations, initial_solution=None):
    bounds = [(V_FY1_MIN, V_FY1_MAX), (0, 2 * np.pi), (1.5, 40), (2, 50)]
    particles_pos = np.random.rand(n_particles, 4)
    for i in range(4):
        particles_pos[:, i] = particles_pos[:, i] * (bounds[i][1] - bounds[i][0]) + bounds[i][0]
    if initial_solution is not None:
        particles_pos[0] = initial_solution
    particles_vel = np.random.randn(n_particles, 4) * 0.1
    pbest_pos = np.copy(particles_pos)
    pbest_fitness = np.array([calculate_fitness(p) for p in pbest_pos])
    gbest_idx = np.argmax(pbest_fitness)
    gbest_pos = pbest_pos[gbest_idx]
    gbest_fitness = pbest_fitness[gbest_idx]
    w, c1, c2 = 0.5, 1.5, 1.5
    print("\n--- 开始优化 ---")
    start_time = time.time()
    for it in range(n_iterations):
        for i in range(n_particles):
            current_fitness = calculate_fitness(particles_pos[i])
            if current_fitness > pbest_fitness[i]:
                pbest_fitness[i] = current_fitness
                pbest_pos[i] = particles_pos[i]
                if current_fitness > gbest_fitness:
                    gbest_fitness = current_fitness
                    gbest_pos = particles_pos[i]
        for i in range(n_particles):
            r1, r2 = np.random.rand(2)
            cognitive_vel = c1 * r1 * (pbest_pos[i] - particles_pos[i])
            social_vel = c2 * r2 * (gbest_pos - particles_pos[i])
            particles_vel[i] = w * particles_vel[i] + cognitive_vel + social_vel
            particles_pos[i] += particles_vel[i]
            for j in range(4):
                particles_pos[i, j] = np.clip(particles_pos[i, j], bounds[j][0], bounds[j][1])
        if (it + 1) % 10 == 0:
            print(f"迭代次数: {it + 1}/{n_iterations}, 当前最优遮蔽时间: {gbest_fitness:.3f} s")
    end_time = time.time()
    print(f"优化完成! 总耗时: {end_time - start_time:.2f} s")
    return gbest_pos, gbest_fitness

# --- 6. 执行与结果 ---
if __name__ == '__main__':
    # --- 第1步: 验证问题1的条件 ---
    print("--- 正在验证问题1的条件 ---")
    v_f_initial = 120.0
    t_d_initial = 1.5
    t_b_initial = 1.5 + 3.6
    direction_vector_xy = TARGET_FALSE[0:2] - P0_FY1[0:2]
    theta_initial = np.arctan2(direction_vector_xy[1], direction_vector_xy[0])
    initial_params = np.array([v_f_initial, theta_initial, t_d_initial, t_b_initial])
    shielding_time = calculate_fitness(initial_params)
    print(f"计算得到的初始遮蔽时间为: {shielding_time:.3f} s")

    # --- 第2步: 以问题1为起点，优化求解问题2 ---
    NUM_PARTICLES = 50
    NUM_ITERATIONS = 150
    best_solution, max_time = pso_optimizer(
        NUM_PARTICLES, 
        NUM_ITERATIONS, 
        initial_solution=initial_params
    )
    
    v_opt, theta_opt, td_opt, tb_opt = best_solution
    drop_point_opt = uav_position(v_opt, theta_opt, td_opt)
    detonation_point_opt = grenade_position(v_opt, theta_opt, td_opt, tb_opt)
    
    print("\n--- 问题2 最优策略 ---")
    print(f"无人机飞行速度 (v_f): {v_opt:.2f} m/s")
    print(f"无人机飞行角度 (θ): {np.rad2deg(theta_opt):.2f} 度")
    print(f"烟幕弹投放点坐标: ({drop_point_opt[0]:.2f}, {drop_point_opt[1]:.2f}, {drop_point_opt[2]:.2f})")
    print(f"烟幕弹起爆点坐标: ({detonation_point_opt[0]:.2f}, {detonation_point_opt[1]:.2f}, {detonation_point_opt[2]:.2f})")
    print(f"烟幕弹爆炸时间: {tb_opt:.3f} s")
    print("--------------------")
    print(f"最大有效遮蔽时间: {max_time:.3f} s")


--- 正在验证问题1的条件 ---
计算得到的初始遮蔽时间为: 1.390 s

--- 开始优化 ---
迭代次数: 10/150, 当前最优遮蔽时间: 2.160 s
迭代次数: 20/150, 当前最优遮蔽时间: 4.080 s
迭代次数: 30/150, 当前最优遮蔽时间: 4.110 s
迭代次数: 40/150, 当前最优遮蔽时间: 4.120 s
迭代次数: 50/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 60/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 70/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 80/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 90/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 100/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 110/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 120/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 130/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 140/150, 当前最优遮蔽时间: 4.130 s
迭代次数: 150/150, 当前最优遮蔽时间: 4.130 s
优化完成! 总耗时: 264.07 s

--- 问题2 最优策略 ---
无人机飞行速度 (v_f): 121.50 m/s
无人机飞行角度 (θ): 179.21 度
烟幕弹投放点坐标: (17617.77, 2.51, 1800.00)
烟幕弹起爆点坐标: (17133.96, 9.19, 1722.29)
烟幕弹爆炸时间: 5.482 s
--------------------
最大有效遮蔽时间: 4.130 s
